# Shopping Trends Analysis – SQL Business Questions

This notebook is focused on using SQL queries to extract answers to business-related questions from the dataset. The business questions are formulated using GPT, and SQL is employed to retrieve the necessary data. In this analysis, the dataset has been imported into a database that is accessed using sqlalchemy. This Python library provides a full power and flexibility of SQL within the Python code. For security, the database credentials are not hardcoded in this notebook; instead, they are securely stored and accessed as environment variables.

## The Dataset

Source: Customer Shopping Trends Dataset (https://www.kaggle.com/datasets/iamsouravbanerjee/customer-shopping-trends-dataset)

The "Customer Shopping Trends Dataset" is a synthetic dataset created to learn more about Data Analysis and Machine Learning, focusing on consumer behavior and purchasing patterns. It contains 3,900 entries, each representing a customer purchase, and includes the following 18 attributes:

- Customer ID: A unique identifier for each customer.
- Age: The customer's age.
- Gender: The gender of the customer (Male/Female).
- Item Purchased: The item that the customer purchased.
- Category: The category of the item purchased (e.g., Clothing, Footwear).
- Purchase Amount (USD): The amount of the purchase in USD.
- Location: The location where the purchase was made.
- Size: The size of the purchased item.
- Color: The color of the purchased item.
- Season: The season during which the purchase was made.
- Review Rating: A rating given by the customer for the purchased item, on a scale of 1 to 5.
- Subscription Status: Indicates if the customer has a subscription (Yes/No).
- Shipping Type: The type of shipping chosen by the customer (e.g., Express, Free Shipping, Next Day Air).
- Discount Applied: Indicates if a discount was applied to the purchase (Yes/No).
- Promo Code Used: Indicates if a promo code was used for the purchase (Yes/No).
- Previous Purchases: The total count of transactions concluded by the customer at the store, excluding the ongoing transaction.
- Payment Method: The customer's most preferred payment method (e.g., Cash, Credit Card, PayPal, Venmo).
- Frequency of Purchases: The frequency at which the customer makes purchases (e.g., Weekly, Fortnightly, Monthly, Annually).

This dataset serves as an excellent foundation for analyzing and deriving insights into consumer preferences, helping businesses tailor their strategies to meet customer needs and enhance satisfaction.

In [1]:
# import the required modules
from sqlalchemy import create_engine    # sqlalchemy connector
import os                               # get the environment variables
import pandas as pd                     # get the pandas

In [2]:
# Access Environment Variables
VSR_HOST = os.getenv('vsr_host')
VSR_USER = os.getenv('vsr_user')
VSR_PASSWORD = os.getenv('vsr_pw')
VSR_DB = os.getenv('vsr_database')

lmubuildengine = create_engine(f'mysql+mysqlconnector://{VSR_USER}:{VSR_PASSWORD}@{VSR_HOST}/{VSR_DB}')

In [3]:
# load the dataset
df = pd.read_csv('shopping_trends.csv')
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [4]:
# renaming columns for easier use in SQL

# rename the columns to include underscores
df.columns = df.columns.str.replace(' ', '_')
# rename 'Purchase Amount (USD)' to Purchase Amount
df.rename(columns={'Purchase_Amount_(USD)': 'Purchase_Amount'}, inplace=True)
df.columns

Index(['Customer_ID', 'Age', 'Gender', 'Item_Purchased', 'Category',
       'Purchase_Amount', 'Location', 'Size', 'Color', 'Season',
       'Review_Rating', 'Subscription_Status', 'Shipping_Type',
       'Discount_Applied', 'Promo_Code_Used', 'Previous_Purchases',
       'Payment_Method', 'Frequency_of_Purchases'],
      dtype='object')

In [5]:
# load the data into the database
df.to_sql('shopping_trends', con=lmubuildengine, if_exists='replace', index=False)

3900

In [6]:
# note: the database has an older version of mysql, hence I cannot make CTEs.
sql = """ SELECT * FROM shopping_trends;  """
pd.read_sql_query(sql, con=lmubuildengine)

,Customer_ID,Age,Gender,Item_Purchased,Category,Purchase_Amount,Location,Size,Color,Season,Review_Rating,Subscription_Status,Shipping_Type,Discount_Applied,Promo_Code_Used,Previous_Purchases,Payment_Method,Frequency_of_Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,No,No,32,Venmo,Weekly
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,No,No,24,Venmo,Quarterly
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,No,No,24,Venmo,Weekly


## SQL Business Questions

### 1. Business Question:

How do purchasing habits vary across different age and gender segments, specifically in terms of transaction volume and average spending, and what strategies can be implemented to enhance sales and customer satisfaction across these segments?

In [7]:
query = """
SELECT
    Gender,
    CASE
        WHEN Age BETWEEN 18 AND 25 THEN '18-25'
        WHEN Age BETWEEN 26 AND 35 THEN '26-35'
        WHEN Age BETWEEN 36 AND 45 THEN '36-45'
        WHEN Age BETWEEN 46 AND 55 THEN '46-55'
        WHEN Age > 55 THEN '56+'
        ELSE 'Under 18'
    END AS Age_Group,
    COUNT(*) AS Total_Purchases,
    AVG(Purchase_Amount) AS Average_Spent
FROM shopping_trends
GROUP BY Gender, Age_Group
ORDER BY Gender, Age_Group, Total_Purchases DESC;
"""

segmentation = pd.read_sql_query(query, con=lmubuildengine)
segmentation

,Gender,Age_Group,Total_Purchases,Average_Spent
0,Female,18-25,173,60.9017
1,Female,26-35,242,62.0620
2,Female,36-45,243,59.2346
3,Female,46-55,246,58.8618
4,Female,56+,344,60.3547
5,Male,18-25,398,60.5377
6,Male,26-35,500,58.6460
7,Male,36-45,486,59.3416
8,Male,46-55,507,61.4181
9,Male,56+,761,58.4678


**Insights:** <br>
1. Gender Differences in Purchasing Volume:
Males in every age group make more purchases than females, with the disparity increasing significantly in the older age groups. Specifically, males aged 56+ make more than twice as many purchases as females in the same age group.

2. Average Spending Consistency:
Average spending is relatively consistent across both genders and all age groups, with amounts typically hovering around the $58 to $62 range. This suggests a similar value perception and purchasing power across different segments.

3. Peak Purchasing Ages:
For females, the highest number of purchases is observed in the 56+ age group, whereas for males, the peak is also in the 56+ age group but with a significantly higher volume.

4. Age Group Differences:
Both genders show an increase in total purchases with age, peaking at the 56+ age group. However, average spending does not follow a clear trend with age, indicating that while older customers may shop more frequently, they do not necessarily spend more per transaction.

**Recommendations:**

1. Targeted Marketing Strategies:
Implement gender and age-targeted marketing campaigns to boost sales in segments with lower transaction volumes. For younger female segments, which show lower purchase volumes, tailored marketing efforts could encourage increased engagement.

2. Loyalty Programs for Older Customers:
Given the high transaction volume among the older segments, particularly males aged 56+, introducing or enhancing loyalty programs targeted at this demographic could further increase customer retention and transaction frequency.

3. Product Assortment and Promotions:
Analyze the product preferences of each segment to tailor the product assortment and promotional offers. Since average spending is consistent, strategies aimed at increasing the average transaction value, such as bundled offers or volume discounts, could be effective across all segments.

4. Enhance Online and In-store Experience for Seniors:
Given the high activity of the 56+ age group, ensuring an age-friendly shopping experience both online and in-store can enhance satisfaction and loyalty. This could include user-friendly website navigation for older users and in-store assistance.

5. Cross-Sell and Upsell Strategies:
Since the average spending per purchase is relatively consistent, there is an opportunity to increase sales through effective cross-selling and upselling strategies. Tailoring these strategies to the preferences and behaviors of each gender and age group can make them more effective.

### 2. Business Question:

How do the usage rates of promo codes and the average purchase amounts vary by location, and what insights can be derived to optimize promotional strategies and sales performance across different regions?

In [8]:
query = """
SELECT
  Location,
  AVG(Purchase_Amount) AS Average_Purchase,
  SUM(CASE WHEN Promo_Code_Used = 'Yes' THEN 1 ELSE 0 END) AS Promo_Codes_Used_Count,
  SUM(CASE WHEN Promo_Code_Used = 'Yes' THEN 1 ELSE 0 END) / COUNT(*) AS Promo_Code_Usage_Rate
FROM shopping_trends
GROUP BY Location
ORDER BY Promo_Code_Usage_Rate DESC;
"""

location_promo = pd.read_sql_query(query, con=lmubuildengine)
location_promo

,Location,Average_Purchase,Promo_Codes_Used_Count,Promo_Code_Usage_Rate
0,Indiana,58.9241,45.0,0.5696
1,Iowa,60.8841,36.0,0.5217
2,Oregon,57.3378,38.0,0.5135
3,South Carolina,58.4079,38.0,0.5000
4,West Virginia,63.8765,40.0,0.4938
5,Missouri,57.9136,40.0,0.4938
6,Hawaii,57.7231,32.0,0.4923
7,Massachusetts,60.8889,35.0,0.4861
8,Oklahoma,58.3467,36.0,0.4800
9,Wisconsin,55.9467,36.0,0.4800


**Insights:**

1. Promo Code Responsiveness: States like Indiana, Iowa, and Oregon have high promo code usage rates, indicating a strong responsiveness to promotions in these areas.

2. Purchase Amounts vs. Promo Usage: There's no clear correlation between high promo code usage and average purchase amounts. For instance, states with lower promo usage like Pennsylvania and Arizona report higher average purchases.

3. Diverse Customer Behavior: The significant variation in promo code effectiveness across states suggests that customer behaviors and preferences towards promotions are diverse.

**Recommendations:**

1. Customize Promotions: Adapt promotional strategies to fit the unique preferences of customers in each state. Regions with high promo code engagement could benefit from increased promotional activities.

2. Alternative Engagement Strategies: In states with low promo code usage but high purchase amounts, explore alternative engagement strategies beyond promo codes to boost sales.

3. Enhance Engagement in Low-Usage States: Investigate and address the reasons behind low promo code usage in certain states, potentially through customer surveys or by testing different promotional messages.

4. Personalize Marketing: Use data analytics for more personalized marketing efforts, tailoring promotions based on customer behavior and regional trends.

5. Agile Strategy Adjustment: Continually monitor the impact of different promotional strategies and be ready to adjust tactics based on performance data to maximize engagement and sales.

### 3. Business Question

What is the distribution of product category preferences within each gender, and how can understanding this distribution help in tailoring marketing and product strategies to better align with the distinct preferences of male and female customers?

In [9]:
query = """
SELECT
    st.Gender,
    st.Category,
    COUNT(*) AS Purchase_Count,
    ROUND((COUNT(*) * 100.0) / gender_total.Total, 2) AS Purchase_Percentage
FROM
    shopping_trends st
JOIN (
    SELECT
        Gender,
        COUNT(*) AS Total
    FROM
        shopping_trends
    GROUP BY
        Gender
) AS gender_total ON st.Gender = gender_total.Gender
GROUP BY
    st.Gender,
    st.Category
ORDER BY
    st.Gender, Purchase_Percentage DESC;
"""

prod_cat = pd.read_sql_query(query, con=lmubuildengine)
prod_cat

,Gender,Category,Purchase_Count,Purchase_Percentage
0,Female,Clothing,556,44.55
1,Female,Accessories,392,31.41
2,Female,Footwear,199,15.95
3,Female,Outerwear,101,8.09
4,Male,Clothing,1181,44.53
5,Male,Accessories,848,31.98
6,Male,Footwear,400,15.08
7,Male,Outerwear,223,8.41


**Insights:**

1. Dominant Category - Clothing: Clothing is the most popular category for both genders, accounting for approximately 44.5% of purchases among both female and male customers. This highlights a universal appeal of clothing items across genders.

2. Second Preference - Accessories: Accessories follow as the second most preferred category, with a slightly higher preference among males (31.98%) compared to females (31.41%). This suggests accessories also hold a significant interest across both genders, albeit with a slight skew towards male customers.

3. Footwear and Outerwear: Footwear and Outerwear are less preferred compared to Clothing and Accessories, yet they maintain a consistent share of purchases across genders. Footwear accounts for about 15% and Outerwear for around 8% of the total purchases for both males and females, indicating a foundational but less emphasized interest in these categories.

4. Gender Parity in Preferences: The distribution of purchase percentages across categories is remarkably similar between genders, suggesting that the overall product category preferences are fairly consistent across male and female customers.

**Recommendations:**

1. Emphasize Clothing in Marketing and Stock: Given the universal appeal of the Clothing category, businesses should prioritize this category in marketing campaigns and ensure a diverse and appealing stock is available to cater to both genders. Highlighting bestselling or trending items within this category can further attract customer interest.

2. Leverage Accessories for Cross-Selling Opportunities: With Accessories being the second most popular category, consider cross-selling strategies that pair accessories with clothing items. For example, suggesting matching accessories with purchased outfits could increase the average order value.

3. Tailored Promotions for Footwear and Outerwear: Although Footwear and Outerwear have lower purchase percentages, they represent essential categories that could benefit from targeted promotions, especially during season changes or specific events that call for these items.

4. Gender-Neutral Marketing Strategies: Given the similarity in category preferences between genders, consider adopting more gender-neutral marketing strategies for the top categories (Clothing and Accessories). This approach can broaden appeal and ensure that marketing efforts resonate across your entire customer base.

5. Feedback and Personalization: Engage customers with surveys or feedback tools to understand specific preferences within these main categories. Use this data to personalize marketing efforts and product recommendations, potentially uncovering niche preferences or underserved areas within these categories.

## Summary Insights:

The analysis conducted provides valuable insights into customer behavior, indicating distinct purchasing patterns across age and gender segments. It highlights the necessity for targeted marketing approaches and product assortments tailored to these specific demographics. Additionally, the variability in promotional strategy effectiveness by location suggests that regional customization of marketing efforts could significantly enhance engagement and sales. Examination of product category preferences also reveals clear gender-specific trends, underlining the importance of aligning marketing and product strategies with male and female customers' distinct preferences. These findings collectively advocate for a data-driven strategy in marketing, product development, and promotional activities to meet the diverse needs and behaviors of the customer base, aiming to enhance customer satisfaction and drive business growth.

----------------------------- THE END ----------------------------------